# spot + api key




In [1]:
import os

# Example from https://pypi.org/project/python-dotenv/
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only

# current directory
cur_dir = os.getcwd()
project_dir = cur_dir.split("/packages/py-quant/try-binance/src/rest")[0]
env_path = Path(project_dir) / ".env"

# load env file
load_dotenv(dotenv_path=env_path)

# api key & secret
api_key = os.environ.get("BINANCE_API_KEY")
api_secret = os.environ.get("BINANCE_API_SECRET")

# print(api_key)
# print(api_secret)

In [ ]:
import os
import time
from binance.spot import Spot
from loguru import logger
import dtale


def get_account():
    # api_key = os.getenv("BINANCE_API_KEY")
    # api_secret = os.getenv("BINANCE_SECRET_KEY")

    assert api_key and api_secret

    # API key/secret are required for user data endpoints
    client = Spot(api_key=api_key, api_secret=api_secret)

    ret = client.account()

    return ret
    for k, v in ret.items():
        if isinstance(v, list):
            for item in v:
                if isinstance(item, dict):
                    if (
                        item.get("free") == "0.00000000"
                        and item.get("locked") == "0.00000000"
                    ):
                        continue
                logger.debug(f"{k}: {item}, type: {type(item)}")
            continue
        logger.debug(f"account info: {k}: {v}")

In [ ]:
import pandas as pd
import dtale

ret = get_account()

# 帐号现货余额信息:
balances = ret.get("balances")

df = pd.DataFrame(balances)

# 筛选掉 free 和 locked 同时为0的数据
df = df[(df["free"] != "0.00000000") | (df["locked"] != "0.00000000")]

# type(df['free'][0])


# 显示余额信息
# df.head()
df.tail()

# dtale.show(df)

,asset,free,locked
712,LDHEI,0.80808496,0.00000000
714,LDKAITO,1.50181254,0.00000000
715,LDRED,2.15809331,0.00000000
718,LDSHELL,1.86990438,0.00000000
720,LDGPS,22.32512465,0.00000000


In [ ]:
#  /api/v3/openOrders
# https://binance-connector.readthedocs.io/en/latest/binance.spot.trade.html


# 现货 - 查看账户当前挂单
def get_open_orders(symbol=None):
    # api_key = os.getenv("BINANCE_API_KEY")
    # api_secret = os.getenv("BINANCE_SECRET_KEY")

    assert api_key and api_secret

    # API key/secret are required for user data endpoints
    client = Spot(api_key=api_key, api_secret=api_secret)

    ret = client.get_open_orders(symbol=symbol)
    return ret


ret = get_open_orders()

In [ ]:
import pandas as pd

df = pd.DataFrame(ret)

# 查看账户当前挂单
df.head()

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
0,DOTFDUSD,38850083,-1,ios_480f5e7dc6bd4109ae06363b705aad4c,12.00000000,50.00000000,0.00000000,0.00000000,NEW,GTC,LIMIT,SELL,0.00000000,0.00000000,1710146283836,1710146283836,True,1710146283836,0.00000000,EXPIRE_MAKER


In [ ]:
# /api/v3/allOrders
# https://binance-connector.readthedocs.io/en/latest/binance.spot.trade.html#all-orders-user-data


def get_all_orders(symbol=None):
    # api_key = os.getenv("BINANCE_API_KEY")
    # api_secret = os.getenv("BINANCE_SECRET_KEY")

    assert api_key and api_secret

    # API key/secret are required for user data endpoints
    client = Spot(api_key=api_key, api_secret=api_secret)

    ret = client.get_orders(symbol=symbol)
    return ret


# 单个交易对, 所有挂单历史
ret = get_all_orders(symbol="DOTFDUSD")

In [15]:
import pandas as pd

df = pd.DataFrame(ret)

# 单个交易对, 所有挂单历史
df.tail()

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
5,DOTFDUSD,18913689,-1,ios_191a1eee214045dfbacb51ef16e7d308,8.20000000,10.00000000,10.00000000,82.00000000,FILLED,GTC,LIMIT,SELL,0.00000000,0.00000000,1704943693280,1704944136323,True,1704943693280,0.00000000,EXPIRE_MAKER
6,DOTFDUSD,19016945,-1,ios_4842e62315f142ac9e39e14a069a2517,8.32000000,10.00000000,10.00000000,83.20000000,FILLED,GTC,LIMIT,SELL,0.00000000,0.00000000,1704958632097,1704958790154,True,1704958632097,0.00000000,EXPIRE_MAKER
7,DOTFDUSD,19207691,-1,ios_c590c76d03b24fdbbacb88ca4d51893a,8.50000000,10.00000000,10.00000000,85.00000000,FILLED,GTC,LIMIT,SELL,0.00000000,0.00000000,1704988297618,1709113738463,True,1704988297618,0.00000000,EXPIRE_MAKER
8,DOTFDUSD,19208635,-1,ios_e7f4c7b606364a5a90e3ba65810e58eb,8.60000000,10.00000000,10.00000000,86.00000000,FILLED,GTC,LIMIT,SELL,0.00000000,0.00000000,1704988403953,1709120035911,True,1704988403953,0.00000000,EXPIRE_MAKER
9,DOTFDUSD,38850083,-1,ios_480f5e7dc6bd4109ae06363b705aad4c,12.00000000,50.00000000,0.00000000,0.00000000,NEW,GTC,LIMIT,SELL,0.00000000,0.00000000,1710146283836,1710146283836,True,1710146283836,0.00000000,EXPIRE_MAKER
